來源 https://github.com/IS2AI/thermal-facial-landmarks-detection

<div><img src="resources/images/ROI.jpg" width="200"/></div>

In [1]:
from imutils import face_utils
from imutils import paths
import numpy as np
import imutils
import dlib
import cv2
import os 

image_path = './test_images/'
model_path = './models/'
upsampling_times = 1


# load the face detector (HOG-SVM)
print("[INFO] loading dlib thermal face detector...")
detector = dlib.simple_object_detector(os.path.join(model_path, "dlib_face_detector.svm"))

# load the facial landmarks predictor
print("[INFO] loading facial landmark predictor...")
predictor = dlib.shape_predictor(os.path.join(model_path, "dlib_landmark_predictor.dat"))

# grab paths to the images
imagePaths = list(paths.list_files(image_path))

gray_list = []

[INFO] loading dlib thermal face detector...
[INFO] loading facial landmark predictor...


In [2]:
# loop over the images
for ind, imagePath in enumerate(imagePaths, 1):

	#ind=1
	print("[INFO] Processing image: {}/{}".format(ind, len(imagePaths)))
	# load the 
	image = cv2.imread(imagePath)

	# resize the image
	image = imutils.resize(image, height=600)

	# copy the image
	image_copy = image.copy()

	# convert the image to grayscale
	image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
	
	# detect faces in the image 

	rects = detector(image, upsample_num_times=upsampling_times)	

	for rect in rects:
		# convert the dlib rectangle into an OpenCV bounding box and
		# draw a bounding box surrounding the face
		(x, y, w, h) = face_utils.rect_to_bb(rect)
		cv2.rectangle(image_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

		# predict the location of facial landmark coordinates, 
		# then convert the prediction to an easily parsable NumPy array
		shape = predictor(image, rect)
		shape = face_utils.shape_to_np(shape)

		next_to_nose_tip = 29
		#取得特定點的顏色
		selected_point_y =  shape[next_to_nose_tip][1]
		selected_point_x =  shape[next_to_nose_tip][0]
		#selected_point = image_copy[selected_point_y, selected_point_x]
		#color = (selected_point[0],selected_point[1],selected_point[2])
		#color = tuple([int(x) for x in color]) 
		gray_color = image[selected_point_y, selected_point_x]
		gray_list.append(gray_color)
		color = (gray_color,gray_color,gray_color)
		color = tuple([int(x) for x in color]) 

		#左側臉頰加點
		point_3_x =  shape[2][0]
		point_3_y =  shape[2][1]
		point_32_x =  shape[31][0]
		point_32_y =  shape[31][1]
		point_left_cheek_x = np.int16(0.7*point_3_x+0.3*point_32_x)
		point_left_cheek_y = np.int16(0.7*point_3_y+0.3*point_32_y)
		shape = np.vstack([shape, [point_left_cheek_x,point_left_cheek_y]])
		point_left_nose_x = np.int16(0.3*point_3_x+0.7*point_32_x)
		point_left_nose_y = np.int16(0.3*point_3_y+0.7*point_32_y)
		shape = np.vstack([shape, [point_left_nose_x,point_left_nose_y]])
		#右側臉頰加點
		point_15_x =  shape[14][0]
		point_15_y =  shape[14][1]
		point_36_x =  shape[35][0]
		point_36_y =  shape[35][1]
		point_right_cheek_x = np.int16(0.7*point_15_x+0.3*point_36_x)
		point_right_cheek_y = np.int16(0.7*point_15_y+0.3*point_36_y)
		shape = np.vstack ([shape, [point_right_cheek_x,point_right_cheek_y]])
		point_right_nose_x = np.int16(0.3*point_15_x+0.7*point_36_x)
		point_right_nose_y = np.int16(0.3*point_15_y+0.7*point_36_y)
		shape = np.vstack ([shape, [point_right_nose_x,point_right_nose_y]])

		#forehead
		point_22_x =  shape[21][0]
		point_22_y =  shape[21][1]
		point_23_x =  shape[22][0]
		point_23_y =  shape[22][1]
		point_28_x =  shape[27][0]
		point_28_y =  shape[27][1]
		point_fore_head_x = np.int16(point_22_x+ point_23_x- point_28_x)
		point_fore_head_y = np.int16(0.75*point_22_y+ 0.75*point_23_y- 0.5*point_28_y)	
		shape = np.vstack([shape, [point_fore_head_x,point_fore_head_y]])	
		
		#throat
		point_9_x =  shape[8][0]
		point_9_y =  shape[8][1]
		point_52_x =  shape[51][0]
		point_52_y =  shape[51][1]
		point_chin_x = np.int16(0.5*point_9_x+0.5*point_52_x)
		point_chin_y = np.int16(0.5*point_9_y+0.5*point_52_y)
		shape = np.vstack ([shape, [point_chin_x,point_chin_y]])	

		if shape[30][0]-shape[27][0]>0:
			point_8_x =  shape[7][0]
			point_8_y =  shape[7][1]
			point_throat_x = np.int16(1.4*point_8_x-0.4*point_52_x)
			point_throat_y = np.int16(1.4*point_8_y-0.4*point_52_y)

		else:
			point_10_x =  shape[9][0]
			point_10_y =  shape[9][1]
			point_throat_x = np.int16(1.4*point_10_x-0.4*point_52_x)
			point_throat_y = np.int16(1.4*point_10_y-0.4*point_52_y)	
		shape = np.vstack ([shape, [point_throat_x,point_throat_y]])
		# loop over the (x, y)-coordinates from our dlib shape
		# predictor model draw them on the image
		point_color = (255, 0, 0)
		point_size = 2
		# for i in range(len(shape)):
		# 	(sx, sy) = shape[i]
		# 	cv2.circle(image_copy, (sx, sy), point_size, point_color, -1)
		index_of_left_eyebrow_outer = 18
		index_of_right_eyebrow_outer = 25
		index_of_left_eyebrow_inner = 21
		index_of_right_eyebrow_inner = 22
		index_of_fore_head = 58
		index_of_upper_nose = 28
		index_of_middle_nose = 29
		index_of_nose_tip = 30
		index_of_left_nostril = 32
		index_of_right_nostril = 34
		index_of_left_nose = 55
		index_of_right_nose = 57
		index_of_left_cheek = 54
		index_of_right_cheek = 56
		index_of_chin = 59
		index_of_throat = 60
		index_of_left_lip = 48
		index_of_upper_lip_outer = 49
		index_of_upper_lip = 52
		index_of_right_lip = 50
		index_of_lower_iip_outer = 51
		index_of_lower_iip = 53
		point_list =[index_of_left_eyebrow_outer,index_of_right_eyebrow_outer,index_of_left_eyebrow_inner,index_of_right_eyebrow_inner,index_of_fore_head,index_of_upper_nose,index_of_middle_nose,index_of_nose_tip,index_of_left_nostril,index_of_right_nostril,index_of_left_nose,index_of_right_nose,index_of_left_cheek,index_of_right_cheek,index_of_chin,index_of_throat,index_of_left_lip,index_of_upper_lip_outer,index_of_upper_lip,index_of_right_lip,index_of_lower_iip_outer,index_of_lower_iip]
		for index in range(len(point_list)):
			cv2.circle(image_copy, shape[point_list[index]], point_size, point_color, -1)			
	

	# show the image

	#image_copy_resize = cv2.resize(image_copy,(300*5,168*5))
	cv2.imshow("Image", image_copy)
	key = cv2.waitKey(0) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"): 
		break

# do a bit cleanup 
cv2.destroyAllWindows()


[INFO] Processing image: 1/2
[INFO] Processing image: 2/2
